In [12]:
import pandas as pd

%load_ext autoreload
%autoreload 2

folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'
csv_folder = '{}/csvs_for_paper'.format(folder)
bed_folder = '{}/bed'.format(folder)
# folder2 = '/projects/ps-yeolab2/obotvinnik/singlecell_pnm'

splicing_feature_folder = '{}/splicing_feature_data'.format(csv_folder)
alternative_feature_folder = '{}/alternative'.format(splicing_feature_folder)
constitutive_feature_folder = '{}/constitutive'.format(splicing_feature_folder)

alt_exons_bedfile = '{}/exons.bed'.format(alternative_feature_folder)
constitutive_bedfile = '{}/exons.bed'.format(constitutive_feature_folder)


import pybedtools

alt_exons = pybedtools.BedTool(alt_exons_bedfile)
constitutive = pybedtools.BedTool(constitutive_bedfile)

from Bio import SeqIO
import kvector
import pybedtools
import pyhomer

DIRECTIONS = 'upstream', 'downstream'

placental_filename = '/projects/ps-yeolab/genomes/hg19/database/phastConsElements46wayPlacental.bed'
conservation_bed = pybedtools.BedTool(placental_filename)

genome = 'hg19'

nt = 200

genome_fasta = '/projects/ps-yeolab/genomes/hg19/chromosomes/all.fa'

pair = pyhomer.ForegroundBackgroundPair(alt_exons, constitutive)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
ls $alternative_feature_folder/*.bed

/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exon2_phastcons_placental_mammal.bed
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons.bed
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons_merkin2012_ancient_alt_exons.bed
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons_phastcons_placental_mammal.bed
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons_stop_plus1.bed
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons_upstream200_foreground.bed
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons_upstream200_placental_foreground.bed
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/test.bed


In [ ]:
! head $alt_exons_bedfile > $alternative_feature_folder/test.bed

Cacluate the time needed for 10 intervals

In [22]:
%%time

import kvector

kmers = kvector.kmer.per_interval_kmers('{}/test.bed'.format(alternative_feature_folder), genome_fasta, conservation_bed)

CPU times: user 1.52 s, sys: 103 ms, total: 1.62 s
Wall time: 3min 32s


```
CPU times: user 1.52 s, sys: 103 ms, total: 1.62 s
Wall time: 3min 32s
```

wow, rounded 4 min per 10 intervals! that's so much! so

In [34]:
%%file ~/count_interval_kmers.py

import sys
import kvector

bedfile = sys.argv[1]
genome_fasta = sys.argv[2]
other = sys.argv[3]

kmers = kvector.kmer.per_interval_kmers(bedfile, genome_fasta, other)
csv = bedfile.replace('.bed', '_kmers.csv')
kmers.to_csv(csv)

Overwriting /home/obotvinnik/count_interval_kmers.py


In [35]:
n_lines = ! wc -l $alt_exons_bedfile
n_lines = int(n_lines[0].split()[0])/10
n_lines

4390

In [36]:
! python ~/count_interval_kmers.py \
    /projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons.bed \
    /projects/ps-yeolab/genomes/hg19/chromosomes/all.fa /projects/ps-yeolab/genomes/hg19/database/phastConsElements46wayPlacental.bed

/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:810: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())
^CTraceback (most recent call last):
  File "/home/obotvinnik/count_interval_kmers.py", line 9, in <module>
    kmers = kvector.kmer.per_interval_kmers(bedfile, genome_fasta, other)
  File "/home/obotvinnik/workspace-git/kvector/kvector/kmer.py", line 159, in per_interval_kmers
    residues=residues).sum()
  File "/home/obotvinnik/workspace-git/kvector/kvector/kmer.py", line 90, in count_kmers
    kmer = str(record[i:(i+k)].seq).upper()
  File "/home/obotvinnik/anaconda/lib/python2.7/site-packages/Bio/SeqRecord.py", line 433, in __getitem__
    des

In [37]:
import qtools

In [38]:
def kmer_command(bedfile, genome_fasta, other):
    return 'python ~/count_interval_kmers.py {} {} {}'.format(bedfile, genome_fasta, other)

bedfiles = alt_exons_bedfile, constitutive_bedfile

for bedfile in bedfiles:
    jobname = bedfile.split('/')[-2] + '_kmers'
    
    commands = []
    
    n_lines = ! wc -l $bedfile
    n_lines = int(n_lines[0].split()[0])
    n_minutes = 4 * n_lines / 10
    
    commands.append(kmer_command(bedfile, genome_fasta, placental_filename))
    
    for direction in DIRECTIONS:
        bed = pybedtools.BedTool(bedfile)
        if direction == 'downstream':
            intron = bed.flank(l=0, r=nt, s=True, genome=genome)
        elif direction == 'upstream':
            intron = bed.flank(l=nt, r=0, s=True, genome=genome)
        intron_filename = bedfile.replace('.bed', '_{}{}nt.bed'.format(direction, nt))
        intron.saveas(intron_filename)
        commands.append(kmer_command(intron_filename, genome_fasta, placental_filename))
#     print commands
    qtools.Submitter(commands, jobname, walltime='0:{}:00'.format(n_minutes), array=True)

running 3 tasks as an array-job.
Wrote commands to alternative_kmers.sh.
Submitted script to queue home.
 Job ID: 5029323
running 3 tasks as an array-job.
Wrote commands to constitutive_kmers.sh.
Submitted script to queue home.
 Job ID: 5029324
